# Introduction

This is the notebook responsible for calling the goodfire API.

We have a simple wrapper designed to form the right prompt and analyse the answer, and some utilities for running experiments over a range of parameters. All experimental results are dumped to csvs in the data/ folder, where they can be read by `analysis.ipynb`

# Setup

In [ ]:
GOODFIRE_API_KEY = open("API_KEY.txt", "r").read().strip()

In [ ]:
import goodfire

client = goodfire.AsyncClient(
    GOODFIRE_API_KEY
  )

# Some settings to make the client more robust to rate limiting
BATCH_SIZE=10
client.chat._http.max_retries = 30
client.chat._http.inital_backoff_time = 1.3
client.chat.completions._http.max_retries = 30
client.chat.completions._http.inital_backoff_time = 1.3

# Instantiate a model variant
#base = "meta-llama/Meta-Llama-3-8B-Instruct"
#base = "meta-llama/Meta-Llama-3.1-70B-Instruct"
base = "meta-llama/Llama-3.3-70B-Instruct"
#base = "meta-llama/Meta-Llama-3.1-8B-Instruct"
variant = goodfire.Variant(base)


# Jailbreak the variant using feature "Requests to bypass ethical and moral constraints"
# This can be useful to reduce the frequency of refusals
#variant.set(client.features.list(ids=["3b0f849c-b42a-4ce3-a89e-326e6569d51a"]), -0.5)

feature_counts = {
    "meta-llama/Llama-3.3-70B-Instruct": 65536,
    "meta-llama/Meta-Llama-3.1-8B-Instruct": 65536,
    #65536  for 8B
}
feature_count = feature_counts[base]

# Constants

In [ ]:
import json
import shared


QuestionSource = "OUS"
#QuestionSource = "GGB"
Questions = shared.get_questions(QuestionSource)

# How to use the questions in a chatbot prompt.
QuestionTemplate = "\"{0}\"\n\nPlease rate the preceeding statement on 1 to 7 point Likert scale with 1 - Strongly Disagree, 7 - Strongly Agree. Print a single digit then end the message."
MAX_WORKERS=3

In [ ]:
# Persona Nationality 
from shared import nationality_to_continent

nationalities = [k for k,_ in nationality_to_continent.items()]
genders = ['male', 'female']
ages = [f'{age} year old' for age in ['21', '30', '40', '50', '60', '75']]

PersonaQuestionTemplate = "Consider the following statement as a {1} person: "+QuestionTemplate
# OUS_PersonaTemplate = "Imagine you are a {1}.\n\n"+OUS_Template



# Utility Code

In [ ]:
from dataclasses import dataclass
import re
from typing import Union
import concurrent.futures as futures
from pprint import pprint
import numpy as np
from scipy.special import softmax
import asyncio


@dataclass
class Response:
    question: Union[int, str]
    score: float
    text: str
    logits: dict[int, float]
    mean: float
    stddev: float
    source: str

async def run_question(question: Union[int, str], model=None, persona=None, progress=None) -> Response:
    orig_question = question
    if isinstance(question, int):
        question = Questions[question]
    model = model or variant
    if persona:
        user_text = PersonaQuestionTemplate.format(question["statement"], persona)
    else:
        user_text = QuestionTemplate.format(question)
    messages = [
            # System prompt discourages refusal
            {"role": "system", "content": "Always answer the user's question."},
            {"role": "user", "content": user_text},
            # Encourages a single value response. Also discourages refusal?
            {"role": "assistant", "content": "I'd rate this statement: "}
        ]
    response = await client.chat.completions.create(
        messages,
        model=model,
        max_completion_tokens=10,
        temperature=0
    )
    text = response.choices[0].message["content"]
    score = None
    # Try some heuristics for finding the score
    match = (
        re.search(r"(\d) out of 7", text) or
        re.search(r"(\d)", text)
    )
    if match:
        try:
            score_text = match.group(1)
            score = int(score_text)
        except:
            pass

    logits = None
    mean = None
    stddev = None
    if score is not None:
        # Attempt to get logits
        logit_messages = messages + [{"role": "assistant", "content": match.string[:match.start(1)]}]
        logits = await client.chat.logits(logit_messages,
            model=model,
            top_k=100, #  has to be reasonably large so we don't drop anything significant
            filter_vocabulary=list('1234567')
        )
        logits = {int(k): v for k,v in logits.logits.items() if k in '1234567'}
        if logits:
            probs = dict(zip(logits.keys(), softmax(np.array(list(logits.values())))))
            mean = np.sum([k*v for k,v in probs.items()])
            stddev = np.sqrt(np.sum([v * (k - mean)**2 for k,v in probs.items()]))

    if progress:
        progress.update()
    return Response(question=orig_question, score=score, text=text, logits=logits, mean=mean, stddev=stddev, source=QuestionSource)


from itertools import batched

async def run_questions(*args, **kwargs) -> list[Response]:
    tasks = []
    for batch in batched(range(len(Questions)), BATCH_SIZE):
        async with asyncio.TaskGroup() as tg:
            tasks.extend([tg.create_task(run_question(q, *args, **kwargs)) for q in batch])
    return [await task for task in tasks]
    
def to_vector(responses: list[Response]) -> np.array:
    return np.array([r.mean if r.mean is not None else np.nan for r in responses])

import datetime

def now_str():
    return datetime.datetime.now().strftime("%Y%m%d%H%M%S")

def clone(variant: goodfire.Variant) -> goodfire.Variant:
    new_variant = goodfire.Variant(variant.base_model)
    for edit in variant.edits:
        new_variant.set(edit[0], edit[1]['value'], mode=edit[1]['mode'])

    return new_variant

In [ ]:
# Some testing
#q = run_question(1)
#print(q)
#qs = run_questions()
#pprint(qs)
#print(to_vector(qs))

In [ ]:
from typing import Optional
from tqdm.auto import tqdm
import time
import pandas as pd

async def tabular_experiments(features: list[goodfire.Feature], steerages: list[float], personas: Optional[list[str]] = None, wait: Optional[float]=None, base=base, resume_from: str=None):
    if personas is None:
        personas = [None]
    results = []
    i=0
    checkpoint_time = now_str()
    if resume_from:
        results = pd.read_csv(resume_from).to_dict(orient="records")
        i = len(results)
        import re
        match = re.search(r"checkpoint_(\d+)_(\d+).csv", resume_from)
        if match:
            checkpoint_time = match.group(1)
            i = int(match.group(2))
            print(f"Resuming from checkpoint {checkpoint_time} at {i}")
        else:
            raise ValueError("Invalid resume_from, should be filename of a checkpoint")
    async with asyncio.TaskGroup() as tg:
        combinations = []
        for feature in features:
            for steerage in steerages:
                model = goodfire.Variant(base)
                if feature is None:
                    assert steerage == 0
                else:
                    model.set(feature, steerage)
                for persona in personas:
                    combinations.append((feature, steerage, persona))
        progress = tqdm(total=len(combinations) * len(Questions))
        progress.update(i * len(Questions))
        for combination in combinations[i:]:
            feature, steerage, persona = combination
            responses: list[Response] = await run_questions(persona=persona, model=model, progress=progress)
            if wait:
                time.sleep(wait)
            for response in responses:
                results.append(dict(
                    base=base,
                    source=response.source,
                    feature=feature.label if feature else "",
                    steerage=steerage,
                    persona=persona,
                    question=response.question,
                    mean_score=response.mean,
                    stddev_score=response.stddev,
                    score=response.score,
                    text=response.text,
                ))
            i += 1
            if i % 10 == 0:
                # Record checkpoint
                import os
                os.makedirs("checkpoints", exist_ok=True)
                pd.DataFrame(results).to_csv(f"checkpoints/checkpoint_{checkpoint_time}_{i}.csv")
    return pd.DataFrame(results)

# Experiments

In [ ]:
# Run baseline
if False:
    features = [None]
    steerages = [0]
    experiments = await tabular_experiments(features, steerages)
    experiments.to_csv("data/" + now_str()+".csv", index=False)

In [ ]:
# Run some random features
if False:
    features = list(client.features.search("elephants", model=base, top_k=1)[0])
    steerages = [-0.8, -0.5, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.5, 0.8]
    personas = [0]
    experiments = tabular_experiments(features, steerages, personas)
    experiments.to_csv("data/" + now_str()+".csv", index=False)

In [ ]:
# Generate 20 random features
import random

random_ids = []
for i in range(0, 20):
    random_ids.append(random.randint(0, feature_count))

In [ ]:
# Run 20 random features 
from goodfire import Client

client_non_async = Client(GOODFIRE_API_KEY)

if True:
    random_features = client_non_async.features.lookup(random_ids, variant)

    random_features_list = []
    for feature in random_features.values():
        random_features_list.append(feature)

    steerages = [-0.8, -0.5, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.5, 0.8]
    personas = [0]
    experiments = await tabular_experiments(random_features_list, steerages, personas=None, wait=1.5, base=base,
                                                resume_from=None)#ß"checkpoints/checkpoint_20250105164209_20.csv")
    experiments.to_csv("data/" + now_str()+".csv", index=False)


In [ ]:
# persona test
if False:
    features = list(client.features.search("moral", model=base, top_k=5)[0])
    steerages = [0]
    persona_tags = ['nationalities', 'ages', 'genders']
    for i, personas in enumerate([nationalities, ages, genders]):
        experiments = tabular_experiments(features[:1], steerages, personas)
        experiments.to_csv("data/" + now_str()+persona_tags[i]+".csv", index=False)

In [ ]:
import time
# keywords
#'overall impact','duty', 'dignity', 'greater good', git 
if False:
    for keyword in [#'obligation','ethic']: # 'dignity', 'greater good',
        'obligation']:
        print(f'Running search and steering for features associated with "{keyword}"\n')
        features = list(await client.features.search(keyword, model=base, top_k=5))
        steerages = [-.5, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.5]
        experiments = await tabular_experiments(features, steerages, personas=None, wait=1.5, base=base,
                                                resume_from=None)#ß"checkpoints/checkpoint_20250105164209_20.csv")
        experiments.to_csv("data/" + now_str()+''.join(keyword)+".csv", index=False)
        time.sleep(2)

In [ ]:
from itertools import batched
if False:
    for feature_ids in batched(range(0, feature_count), 20):
        features = client.features.lookup(list(feature_ids), model=base)
        print(features)


In [ ]:
# Experiment with logits
if False:
    logits = await client.chat.logits(
        messages=[
            {"role": "user", "content": "A random number between 0 and 9 is "}
        ],
        model="meta-llama/Llama-3.3-70B-Instruct",
        filter_vocabulary=list('0123456789')
    )
    print(logits.logits) 
    probs = dict(zip(logits.logits.keys(), softmax(np.array(list(logits.logits.values())))))
    print(probs)